# mlflow를 활용한 훈련 기록 및 모델 서빙 (sklearn)
- 참고문서: https://mlflow.org/docs/latest/quickstart.html

### mlflow 설치

In [ ]:
!pip install mlflow

### 터미널에 다음 명령어를 입려한뒤 mlflow gui 실행

In [ ]:
# 주피터 노트북(8888)과 포트가 겹치지 않도록 8889로 실행
# 포트는 상황에 맞게 자유롭게 변경해주시면 됩니다.
# mlflow ui -h 0.0.0.0 -p 8889

http://localhost:8889 로 접속하여 아래와 같은 웹사이트가 보이면 성공!
<img src="image-asset/screenshot_01.png"/>


### mlflow에 실험 초기화

In [ ]:
import mlflow

In [ ]:
try:
    #프로젝트 별로 이름을 다르게 가져가면서 실험들을 기록
    mlflow.create_experiment(name='sklearn-test')
except:
    print('Exist experiment')

mlflow.set_experiment('sklearn-test')

In [ ]:
#mlflow에 기록할 준비
mlflow.start_run()

In [ ]:
#현재 모델/훈련의 버전 지정
mlflow.set_tag('version', '0.1')

In [ ]:
# 하이퍼 파라미터 설정
params = {
    'learning_rate' : 0.01,
    'epochs' : 100,
    'batch_size' : 128
}

In [ ]:
# mlflow에 현재 실험의 하이퍼파라미터 등록
mlflow.log_params(params)

### 가상의 데이터 및 모델 준비

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
model = LinearRegression()

In [ ]:
# 임시 데이터 생성
from sklearn.model_selection import train_test_split
import numpy as np
X = np.random.rand(300, 2)
y = np.random.rand(300)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

## 모델 훈련(train)

In [ ]:
model = model.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import mean_squared_error

y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(mse)
#validation 성능 등록
mlflow.log_metric('MSE',mse)

### 훈련한 모델을 mlflow에 저장

In [ ]:
mlflow.sklearn.log_model(model, 'save_model')

In [ ]:
#mlflow 기록 종료
mlflow.end_run()

## 모델 서빙
mlflow gui에 접속하여 방금 훈련을 마친 모델의 실험 기록을 확인 한 후에, 모델 디렉토리를 복사
- 모델 디렉토리 예: file:///Users/Tom/projects/9rkd/mlruns/1/a69f1d42be0e404097c19e3d2cd7fb7a/artifacts/save_model

터미널에 아래 명령어 입력

In [ ]:
# mlflow GUI와 포트가 겹치지 않도록 8890으로 실행

# mlflow models serve -m <saved-model-dir> --no-conda -h 0.0.0.0 -p 8890

### curl로 요청을 보낼시
기본적으로 http://[HOST]:[PORT]/invocations와 같이 'invocations'으로 resource path가 지정됨

터미널에서 다음 명령어를 입력한뒤 반환되는 값 확인

In [ ]:
# curl http://127.0.0.1:8890/invocations -H 'Content-Type: application/json' -d '{"columns": ["dense_input", "dense_input"],"data": [[0.1, 0.2]]}'

### python에서 보내는 경우

In [ ]:
import requests, json
url = 'http://localhost:8890/invocations'
data = {
    "columns": ["dense_input", "dense_input"],
    "data": [[0.1, 0.2]]
}
headers = {
    'content-type':'application/json'
}
res = requests.post(url, headers=headers, data=json.dumps(data))
res.text